# CrewAI Notes

The goal is to build out a couple of crews to do the following: 

1. Job Application Crew: I am tired of look and applying for jobs, so I want the crew to do the heavy lifting
2. Math Tutor Helper: the goal is to help me come up with a curriculum on the fly for math tutees
3. Blog Article Crew: to do the research on trending topics and do the heavy lifting. 

These are my notes from the Deeplearning.ai course, [Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/eusjw/create-agents-to-research-and-write-an-article-(code)). our 

__Go to this website to get the [crewAI badge](https://learn.crewai.com) when you get done with the course!!!__

My setup: 
Python 3.12.9, Conda env

Bring in the libraries:

`pip install crewai crewai_tools langchain_community`

In [58]:
from crewai import Agent, Task, Crew, LLM

import os

##### A YT video on using CrewAI 

Here's the chick's git repository and references she uses: 
https://github.com/majacinka/crewai-experiments/blob/main/google_newsletter.py

https://www.promptingguide.ai/techniques/tot

https://www.youtube.com/watch?v=kJvXT25LkwA

Got these errors:

Running cells with '.conda (Python 3.11.11)' requires the ipykernel package.
Run the following command to install 'ipykernel' into the Python environment.
Command: 'conda install -n .conda ipykernel --update-deps --force-reinstall'


Ran pip to install ipykernel, got it from this [issue resolution](https://github.com/microsoft/vscode-jupyter/issues/15544): 

- `pip install ipykernel -U --user --force-reinstall`

Utils not installed : https://unbiased-coder.com/solve-no-module-named-utils/

- `pip install utils`

In [ ]:
import os 
from langchain_openai import ChatOpenAI
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


In [ ]:
# https://community.deeplearning.ai/t/l2-import-error/650750/12

# #this does work if you ever want to use chatgpt. need to create a .env file & then add it to the .gitignore file so that you dont share your api key

# from dotenv import load_dotenv

# load_dotenv()
# client = OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY") 
# )

# Using Ollama
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "testapikey"

llm_local = ChatOpenAI(
model="ollama/llama3.2",
base_url="http://localhost:11434"
)

I used the comments from this [thread](https://community.crewai.com/t/connecting-ollama-with-crewai/2222/9) to figure out how to use Ollama instead of ChatGPT. 

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

#### Using Ollama

You have to add some time limits and max iterations, because you can max out your system's RAM. 

```
max_iterations=5,
max_time=120,
```

The max_iterations or max_iter attribute in CrewAI is used to define the maximum number of iterations an agent can perform for a single task. This feature helps prevent infinite loops or excessively long executions.

Key points about max_iterations:

Default value: The default is set to 25 iterations, balancing thoroughness and efficiency.

Purpose: It limits how many times an agent will attempt to complete a task before giving its best answer.

Usage: You can set this attribute when initializing an agent. For example:

```
agent = Agent(
  role='Research Analyst',
  goal='Provide up-to-date market analysis',
  backstory='An expert analyst with a keen eye for market trends.',
  tools=[search_tool],
  max_iter=25,  # Set maximum iterations
)
```

Behavior: As the agent approaches the maximum number of iterations, it will try to provide the best possible answer based on its current progress.

Customization: You can adjust this value based on your specific needs, allowing for more or fewer attempts depending on the complexity of the task.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm_local, # this is only needed if using a local LLM, otherwise remove
    allow_delegation=False,
    max_iterations=5,
    max_time=120,
	verbose=True
)

writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm_local, # this is only needed if using a local LLM, otherwise remove
    allow_delegation=False,
    max_iterations=5,
    max_time=120,
    verbose=True
)

editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm_local, # this is only needed if using a local LLM, otherwise remove
    allow_delegation=False,
    max_iterations=5,
    max_time=120,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [68]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [69]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [70]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents so they can act as a unit.
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [77]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit], # order is important for this crew
    # verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [72]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Comprehensive Content Plan for Blog Article on Artificial Intelligence

I. Introduction
1. Definition of Artificial Intelligence (AI)
2. Brief overview of AI's impact on modern society
3. Importance of understanding AI for informed decision-making

II. Latest Trends in Artificial Intelligence
1. Advancements in Machine Learning (ML) and Deep Learning (DL)
    a. Applications: Computer Vision, Natural Language Processing (NLP), and Predictive Analytics
    b. Key Players: Google, Facebook, and Microsoft
2. Increased Focus on Explainable AI (XAI)
    a. Importance of transparency in AI decis

- Display the results of your execution as markdown in the notebook.

In [76]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

The provided text appears to be a comprehensive overview of artificial intelligence (AI), its trends, applications, and implications. The content covers various aspects of AI, including Explainable AI (XAI), Edge AI, key players in the field, noteworthy news in AI-powered healthcare and cybersecurity, and target audience analysis.

To improve the text, I suggest adding more specific examples and anecdotes to make it more engaging and relatable. Additionally, incorporating visual elements such as images or diagrams can help illustrate complex concepts and break up the dense technical content.

Furthermore, the text could benefit from a clearer structure and organization. Consider breaking down the content into clear sections or categories, such as "Trends in AI," "Applications of AI," "Key Players in AI," and so on. This would make it easier for readers to navigate the content and understand the relationships between different topics.

Finally, consider adding more concrete calls-to-action or suggestions for further learning and exploration. While the text mentions online courses, books, and websites, it would be helpful to provide more specific recommendations or resources for readers who want to dive deeper into AI-related topics.

Here is a revised version of the text incorporating some of these suggestions:

**The Future of Artificial Intelligence: Trends, Applications, and Implications**

Artificial Intelligence (AI) has revolutionized modern society, transforming the way we live, work, and interact with each other. As AI continues to advance, it's essential to understand its latest trends, applications, and implications on our daily lives.

**Trends in AI**

One of the most significant advancements in AI is the growing focus on Explainable AI (XAI). XAI aims to provide transparency and accountability in AI decision-making, ensuring that algorithms are fair, reliable, and trustworthy. For example, Google's recent breakthrough in XAI has enabled the development of more accurate and transparent predictive models.

Another trend gaining traction is Edge AI, which enables AI applications to run on devices at the edge of the network, reducing latency and improving security. Applications include IoT, autonomous vehicles, and smart homes. Consider this: imagine being able to control your smart home devices with just your voice, thanks to Edge AI-powered sensors and algorithms.

**Applications of AI**

AI-powered healthcare is transforming the medical landscape, with applications in medical imaging analysis, patient monitoring, and personalized medicine. For instance, IBM's Watson for Genomics has helped researchers identify new genetic markers for diseases such as cancer. Similarly, AI-driven cybersecurity systems can detect and respond to threats in real-time, protecting sensitive data and preventing cyber-attacks.

In education, AI is revolutionizing the way we learn, with applications in personalized learning, intelligent tutoring systems, and automated grading. Consider this: imagine having a virtual tutor that adapts to your learning style and pace, providing you with tailored feedback and guidance. This is made possible by AI-powered educational platforms like DreamBox Learning.

**Key Players in AI**

Google DeepMind continues to push the boundaries of AI research, making breakthroughs in areas like AlphaGo, AlphaFold, and LaMDA. Microsoft Azure Machine Learning provides a cloud-based platform for building, deploying, and managing AI models, while Amazon SageMaker offers a fully managed service for machine learning model development.

**Noteworthy News in AI**

AI-powered cybersecurity is gaining momentum, with emerging technologies like AI-powered security tools and automation platforms. For example, Google's recent acquisition of DeepMind has enabled the company to develop more advanced AI-powered security systems that can detect and respond to threats in real-time.

In education, AI-driven adaptive learning platforms are changing the way we learn, providing personalized feedback and guidance to students. Consider this: imagine having a virtual coach that helps you improve your skills and achieve your goals, using AI-powered analytics and feedback loops.

**Target Audience Analysis**

Our target audience consists of professionals aged 25-45 years old, working in technology, business, and education. They are interested in AI and machine learning, as well as technology and innovation. However, they often struggle with limited understanding of AI and its applications, making it essential to provide resources for further learning and professional development.

**Call to Action**

We encourage readers to explore AI-powered solutions for their businesses or industries. Consider taking online courses like Coursera's Machine Learning course by Andrew Ng, or reading books like "Life 3.0: Being Human in the Age of Artificial Intelligence" by Max Tegmark. Join the conversation on websites like AI Today, Towards Data Science, and KDnuggets to stay up-to-date with the latest developments in AI.

**SEO Keywords and Relevant Data**

Primary keywords: Artificial Intelligence (AI), Machine Learning (ML), Deep Learning (DL), Explainable AI (XAI)
Secondary keywords: Computer Vision, Natural Language Processing (NLP), Predictive Analytics
Relevant data sources:
* Pew Research Center: "Artificial Intelligence Trends and Applications"
* McKinsey Global Institute: "A Future That Works: Automation, Artificial Intelligence, and the Threat to Jobs"
* IEEE Robotics & Automation Society: AI-related publications and research papers

By incorporating more specific examples, anecdotes, and visual elements, this revised version of the text aims to make it more engaging, relatable, and informative for readers.

 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function

### For using Llama locally 

with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

See above. 